In [68]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import sklearn

In [69]:
'''Загрузите выборку'''
df = pd.read_csv('wine.data', header=None)

In [70]:
"""
Извлеките признаки и классы.
Класс записан в первом столбце.
Признаки - со второго по последний.
"""

y = df[0] # класс
X = df.loc[:, 1:] # Признаки. Выводим все строчки; выводим столбцы, начиная со второго

In [71]:
"""
Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). 
Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). 
Для воспроизводимости результата создавайте генератор KFold с фиксированным параметром random_state=42. 
В качестве меры качества используйте долю верных ответов (accuracy).
"""

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [72]:
import numpy as np
"""
Найдите точность классификации на кросс-валидации для метода k ближайших соседей
(sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество?
Чему оно равно (число в интервале от 0 до 1)? 
"""


def test_accuracy(kf, X, y):
    cv_scores = []
    
    for k in range(1, 51):
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X, y, cv=kf, scoring='accuracy')
        cv_scores.append(scores.mean())
        
    print(np.array(cv_scores))
    
    best_score = max(cv_scores)
    optimal_k = cv_scores.index(best_score) + 1
    return optimal_k, best_score


test_accuracy(kf, X, y)

[ 0.73047619  0.66253968  0.70825397  0.65777778  0.67460317  0.67428571
  0.68        0.68        0.70238095  0.68015873  0.70253968  0.69666667
  0.69095238  0.67936508  0.70190476  0.67952381  0.7015873   0.67952381
  0.67936508  0.69095238  0.70190476  0.69650794  0.70206349  0.70761905
  0.70190476  0.69650794  0.69634921  0.70793651  0.71349206  0.70793651
  0.69095238  0.71349206  0.71349206  0.72460317  0.72460317  0.71349206
  0.71349206  0.71349206  0.70793651  0.70777778  0.71349206  0.70777778
  0.70777778  0.70777778  0.70777778  0.70777778  0.69666667  0.71904762
  0.69666667  0.70777778]


(1, 0.7304761904761905)

In [73]:
"""
Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale.
Снова найдите оптимальное k на кросс-валидации.
Какое значение k получилось оптимальным после приведения признаков к одному масштабу?
Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?
"""


from sklearn.preprocessing import scale
X = scale(X)
test_accuracy(kf, X, y)

[ 0.94396825  0.93285714  0.95507937  0.93825397  0.94936508  0.94952381
  0.94952381  0.9552381   0.96079365  0.96079365  0.96095238  0.9552381
  0.94952381  0.96634921  0.97206349  0.97206349  0.96650794  0.96650794
  0.95539683  0.96650794  0.96095238  0.96650794  0.96095238  0.9552381
  0.9552381   0.96079365  0.9552381   0.96634921  0.97761905  0.96079365
  0.9552381   0.96079365  0.96634921  0.96634921  0.96079365  0.96079365
  0.9552381   0.96079365  0.96079365  0.96079365  0.96634921  0.96079365
  0.96634921  0.96063492  0.96634921  0.94952381  0.95507937  0.94952381
  0.95507937  0.96063492]


(29, 0.9776190476190475)

## Подбор параметра метрики

Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным подходом является
подбор метрики под конкретную задачу.

Здесь берётся за основу метрика Минковского.

In [74]:
from sklearn import datasets
from sklearn.preprocessing import scale # Для масштабирования признаков
from numpy import linspace
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import KFold, cross_val_score

/Users/user/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [75]:
df = datasets.load_boston()

In [76]:
# Признаки записаны в поле data, а целевой вектор — в поле target.
X = df['data'] # Признаки
y = df['target'] # Целевой вектор

In [77]:
# Приводим признаки к одному масштабу
X = scale(X)

In [78]:
"""Переберите разные варианты параметра метрики p по сетке от 1 до
10 с таким шагом, чтобы всего было протестировано 200 вариантов"""

best_p = None
best_score = None
for p in linspace(1, 10, 200):
    clf = KNeighborsRegressor(n_neighbors=5, weights='distance')
    clf.fit(X, y)

    kf = KFold(len(y), n_folds=5, random_state=42)
    scores = cross_val_score(clf, X, y, cv=kf, scoring='neg_mean_squared_error')

    if best_score is None or best_score < scores.mean():
        best_p = p
        best_score = scores.mean()

In [79]:
"""Определите, при каком p качество на кросс-валидации оказалось
оптимальным"""
print(best_p)

1.0
